# The InvestmentFlowsIndia data-set

The python version of this sample can be found [here](https://github.com/shyams80/plutons/blob/master/docs-py/InvestmentFlowsIndia.ipynb)

In [1]:
library(tidyverse)
library(odbc)
library(plutoR)
options("scipen"=999)
source("config.R")

#initialize
iflows <- InvestmentFlowsIndia()

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.0     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.1
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


### get daily DII and FII flows for the last 20 days for the cash market

In [2]:
stDate <- Sys.Date() - 30

diiCash <- iflows$DiiCashMarket() %>%
  group_by(TIME_STAMP, SECURITY_TYPE) %>%
  summarize(BUY = sum(BUY_VALUE),
            SELL = sum(SELL_VALUE),
            NET = sum(BUY_VALUE - SELL_VALUE)) %>%
  filter(TIME_STAMP >= stDate) %>%
  select(TIME_STAMP, SECURITY_TYPE, BUY, SELL, NET)

fiiCash <- iflows$FiiCashMarket() %>%
  group_by(TIME_STAMP, SECURITY_TYPE) %>%
  summarize(BUY = sum(BUY_VALUE),
            SELL = sum(SELL_VALUE),
            NET = sum(BUY_VALUE - SELL_VALUE)) %>%
  filter(TIME_STAMP >= stDate) %>%
  select(TIME_STAMP, SECURITY_TYPE, BUY, SELL, NET)

diiCash %>%
  full_join(fiiCash, by=c('TIME_STAMP', 'SECURITY_TYPE')) %>%
  collect() %>%
  mutate(BUY = replace_na(BUY.x, 0) + replace_na(BUY.y, 0),
         SELL = replace_na(SELL.x, 0) + replace_na(SELL.y,0),
         NET = replace_na(NET.x,0) + replace_na(NET.y,0)) %>%
  select(TIME_STAMP, SECURITY_TYPE, BUY, SELL, NET) %>%
  print(n=Inf)

Warning message:
“Missing values are always removed in SQL.
Use `SUM(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”

# A tibble: 63 x 5
# Groups:   TIME_STAMP [21]
   TIME_STAMP SECURITY_TYPE      BUY     SELL        NET
   <date>     <chr>            <dbl>    <dbl>      <dbl>
 1 2019-06-24 Debt           9671.    9660.      11.4   
 2 2019-06-24 Equity        13542.   13285.     257.    
 3 2019-06-24 Hybrid            5.38     5.43    -0.0500
 4 2019-06-25 Debt           9698.    8810.     887.    
 5 2019-06-25 Equity         6216.    5213.    1003.    
 6 2019-06-25 Hybrid            4.77     2.83     1.94  
 7 2019-06-26 Debt           7569.    7559.      10.7   
 8 2019-06-26 Equity         8637.    5689.    2948.    
 9 2019-06-26 Hybrid          347.      12.0    335.    
10 2019-06-27 Debt          16941.   14805.    2136.    
11 2019-06-27 Equity        10339.   10381.     -41.3   
12 2019-06-27 Hybrid         1650.       1.41  1649.    
13 2019-06-28 Debt          10914.   12647.   -1733.    
14 2019-06-28 Equity         9320.    9057.     263.    
15 2019-06-28 Hybrid          265.      5

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)